# Segmenting and Clustering Neighborhoods in Toronto - Notebook 02

## Importing the Pandas library:

In [ ]:
import pandas as pd

## Reading the data from the URL and selecting the first returned table:

In [ ]:
postal_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

## Naming the columns accordingly to what is expected:

In [ ]:
postal_df.columns = ['PostalCode', 'Borough', 'Neighborhood']

## A first look at our data:

In [ ]:
postal_df.head(10)

## Removing the rows with invalid borough entries: 

In [ ]:
postal_df = postal_df[postal_df.Borough != 'Not assigned']

postal_df.reset_index(drop = True, inplace = True)

## Correcting the invalid neighborhood entries:

In [ ]:
for i in range(len(postal_df)):
    
    if postal_df.loc[i, 'Neighborhood'] == 'Not assigned':
    
        postal_df.loc[i, 'Neighborhood'] = postal_df.loc[i, 'Borough']

## Looking at our dataframe with the corrected values for boroughs and neighborhoods:

In [ ]:
postal_df.head(10)

## Creating a new dataframe to contain the grouped neighborhood entries:

In [ ]:
output_df = postal_df.drop_duplicates(['PostalCode', 'Borough']).copy()

output_df.reset_index(drop = True, inplace = True)

for i in range(len(output_df)):
    
    nh_list = []
    
    for j in range(len(postal_df)):
    
        if postal_df.loc[j, 'PostalCode'] == output_df.loc[i, 'PostalCode']:
        
            if postal_df.loc[j, 'Borough'] == output_df.loc[i, 'Borough']:
            
                nh_list.append(postal_df.loc[j, 'Neighborhood'])
    
    nh_string = nh_list[0]
    
    for k in range(1, len(nh_list)):
    
        nh_string = nh_string + ', ' + nh_list[k]
    
    output_df.loc[i, 'Neighborhood'] = nh_string    

## Looking at the first ten results of our operations:

In [ ]:
output_df.head(10)

## Checking the number of rows of our dataframe with grouped neighborhoods: 

In [ ]:
print("Number of rows: {}".format(output_df.shape[0]))

## Downloading the geospatial database:

In [ ]:
!wget -O Geospatial_data.csv https://cocl.us/Geospatial_data

## Loading the geospatial database into a Pandas dataframe and checking the data:

In [ ]:
geospatial_df = pd.read_csv('Geospatial_data.csv')

geospatial_df.columns = ['PostalCode', 'Latitude', 'Longitude']

geospatial_df.head()

## Creating two new columns in our main dataframe:

In [ ]:
output_df['Latitude'] = pd.Series()

output_df['Longitude'] = pd.Series()

## Filling the new columns with data gathered from the geospatial dataframe: 

In [ ]:
for i in range(len(output_df)):
    
    code = output_df.loc[i, 'PostalCode']
    
    latitude = geospatial_df.loc[geospatial_df.PostalCode == code]['Latitude'].values[0]
    
    longitude = geospatial_df.loc[geospatial_df.PostalCode == code]['Longitude'].values[0]
    
    output_df.loc[i, 'Latitude'] = latitude
    
    output_df.loc[i, 'Longitude'] = longitude

## Checking the results of our operations:

In [ ]:
output_df.head(10)